# `TEANAPS` NEWS ARTICLE SCRAPPER
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

> **\*\*\* 주의사항 \*\*\***  
본 자료에서 설명하는 웹크롤링하는 방법은 해당 기법에 대한 이해를 돕고자하는 교육과 이를 활용한 연구 목적으로 사용되었으며, 대량의 무단 크롤링 및 상업적 활용을 금합니다.

## 1. 데이터 수집 준비하기

### 1-1. Selenium 라이브러리 설치

In [1]:
# 가상의 브라우저를 컨트롤 할 수 있도록 도와주는 selenium 패키지를 설치합니다.
# 아래 주석을 해지하고 셀을 실행합니다.
# 설치는 한번만 수행하면 되며, 재설치시 Requirement already satisfied: ~ 라는 메시지가 출력됩니다.

#!pip install selenium

### 1-2. Chrome Driver 다운로드 및 경로설정

- Chrome Driver 다운로드 URL : http://chromedriver.chromium.org/downloads

In [2]:
# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

# Windows 운영체제
#DRIVER_PATH = "chromedriver.exe"

# MAC, Linux 운영체제
# - 경고메시지 출력 시 보안 및 개인정보보호 메뉴에서 "확인없이 허용" 클릭 필요함
DRIVER_PATH = "./chromedriver"

### 1-3. 라이브러리 Import 및 Chrome Driver 실행

In [3]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [4]:
# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
# Chrome Driver를 호출합니다.
driver = webdriver.Chrome(executable_path = DRIVER_PATH, options=chrome_options)

## 2. 카페 게시글 및 댓글 수집하기

---

### 2-1. 카페 검색결과 정보 입력

In [5]:
# 수집할 카페 검색결과 정보를 입력합니다.
                                  # 네이버 카페에서 게시글을 검색하고 검색결과 URL을 입력합니다.
                                  # URL은 네이버 카페에서 게시글 검색 후 검색결과 페이지 번호의 링크를 복사하여 찾을 수 있습니다.
                                  # 전체 URL에서 맨 뒤 페이지 번호는 제거하고 입력합니다. (URL = "~search.page=")
URL = "https://cafe.naver.com/dieselmania?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=11262350%26search.media=0%26search.searchdate=all%26userDisplay=15%26search.option=0%26search.sortBy=date%26search.searchBy=0%26search.query=%C3%BB%B9%D9%C1%F6%26search.viewtype=title%26search.page="
PAGE_LIMIT = 3                    # 검색결과 중 수집할 최대 페이지 수를 입력합니다. (검색결과 페이지 수 보다 적게 입력)
POST_LIMIT = 5                    # 수집할 최대 게시글 수를 입력합니다.
SAVE_FILENAME = "cafe_post.txt"   # 수집결과를 저장할 파일명을 입력합니다.

### 2-2. 검색결과 게시글 URL 수집

In [6]:
# 수집할 카페 게시물의 링크주소를 저장할 리스트를 생성합니다.
post_list = []

# FOR 문을 활용해 페이지 번호를 반복합니다.
for page_num in range(1, PAGE_LIMIT+1):
    # 검색결과 페이지로 이동합니다.
    driver.get(URL + str(page_num))
    # 페이지에서 게시물 리스트가 포함된 프레임으로 이동합니다.
    driver.switch_to_frame(driver.find_element(By.NAME, "cafe_main"))
    # 게시물 태그를 모두 불러옵니다.
    elem = driver.find_elements(By.CLASS_NAME, "article")
    for e in elem:
        # 웹페이지의 하이퍼링크 URL은 항상 href 속성에 존재합니다.
        # href 속성에 저장된 URL을 불러와 post_list에 추가합니다.
        post_list.append(e.get_attribute("href"))
    # 아래 주석부분은 위 25~29번째 라인과 동일한 코드입니다.
    # 혹시 post_list에 URL이 저장되지 않는 경우 아래처럼 태그의 Class 속성을 "article"에서 "aaa"로 변경해줍니다.
    '''
    elem = driver.find_elements(By.CLASS_NAME, "aaa")
    for e in elem:
        post_list.append(e.find_element(By.TAG_NAME, "a").get_attribute("href"))
    '''
    # 페이지의 기본 프레임으로 이동합니다.
    driver.switch_to_default_content() 
    time.sleep(2)

<ipython-input-6-e015faad37a6>:9: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(driver.find_element(By.NAME, "cafe_main"))
<ipython-input-6-e015faad37a6>:24: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()


In [7]:
# 총 몇개의 게시물 URL이 저장되었는지 확인합니다.
print("수집된 게시물 URL 개수 :", len(post_list))

수집된 게시물 URL 개수 : 45


In [8]:
# URL이 잘 저장되어 있는지 인덱싱을 통해 일부만 확인해봅니다.
print(post_list[:1])

['https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=11262350&page=1&inCafeSearch=true&searchBy=1&query=%EC%B2%AD%EB%B0%94%EC%A7%80&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=36096011&referrerAllArticles=true']


In [9]:
print(post_list[0])

https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=11262350&page=1&inCafeSearch=true&searchBy=1&query=%EC%B2%AD%EB%B0%94%EC%A7%80&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=36096011&referrerAllArticles=true


### 2-3. 게시글 내용 및 댓글 수집

In [10]:
# 게시물 내용을 저장할 파일을 생성합니다.
f = open(SAVE_FILENAME, "w", encoding="utf-8")
progress = 0

# FOR 문을 활용해 페이지 URL을 반복합니다.
for url in post_list[:POST_LIMIT]:
    # 현재 수집이 진행중인 인덱스를 출력합니다.
    progress += 1
    print(str(progress) + "/" + str(len(post_list)), end="\r")
    
    try:
        # URL을 통해 게시물 페이지로 이동합니다.
        driver.get(url)
        time.sleep(5)
        # 페이지에서 게시물 내용이 포함된 프레임으로 이동합니다.
        driver.switch_to.frame(driver.find_element(By.NAME, "cafe_main"))

        # 페이지에서 태그의 속성을 활용해 원하는 부분을 불러옵니다.
        # 1. 게시물 제목
        title = driver.find_element(By.CLASS_NAME, "title_text").text.strip()
        #title = elem.find_elements(By.TAG_NAME, "td")[0].text
        # 2. 게시판 이름
        board_type = driver.find_element(By.CLASS_NAME, "link_board").text.strip()
        # 3. 게시물 작성일자
        datetime = driver.find_element(By.CLASS_NAME, "date").text
        # 4. 게시물 작성자(닉네임)
        user = driver.find_element(By.CLASS_NAME, "nickname").text
        # 5. 게시물 내용
        # 파일에서 게시물 구분을 줄단위로 하기위해 줄바꿈을 모두 제거합니다.
        contents = driver.find_element(By.CLASS_NAME, "article_viewer").text.replace("\n", " ")
        # 파일에 수집한 게시물 내용을 기록합니다.
        f.write("POSTING" + "\t" + user + "\t" + datetime + \
                "\t" + board_type + "\t" + title + "\t" + contents + "\n") 

        # 댓글을 모두 불러옵니다.
        reply_list = driver.find_elements(By.CLASS_NAME, "comment_box")
        # FOR 문을 활용해 댓글을 모두 반복합니다.
        for reply in reply_list:
            # 6. 댓글 내용
            comment = reply.find_element(By.CLASS_NAME, "text_comment").text.replace("\n", " ")
            # 7. 댓글 작성자(닉네임)
            user_nik = reply.find_element(By.CLASS_NAME, "comment_nickname").text
            # 8. 댓글 작성일자
            reply_date = reply.find_element(By.CLASS_NAME, "comment_info_date").text
            # 파일에 수집한 댓글 내용을 기록합니다.
            f.write("COMMENT" + "\t" + user_nik + "\t" + reply_date + \
                    "\t" + board_type + "\t" + "None" + "\t" + comment + "\n")
        # 페이지의 기본 프레임으로 이동합니다.
        driver.switch_to.default_content()
        time.sleep(2)
    except:
        driver.switch_to.default_content()
        time.sleep(2)
        continue
f.close()

<ipython-input-10-04ad9ee120a7>:16: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(driver.find_element(By.NAME, "cafe_main"))
<ipython-input-10-04ad9ee120a7>:49: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()


In [11]:
# Chrome Driver를 닫습니다.
driver.close()

In [12]:
# 파일에 저장된 카페 게시글 내용을 확인합니다.
f = open(SAVE_FILENAME, encoding="utf-8")
for post in f:
    print(post.strip())
f.close()

POSTING	백구윈터	2021.10.02. 19:56	■디매인 라이프■	문제의 와이드 청바지 반품 신청했습니다	많은 분들의 피드백 반영하여 결국엔 반품 선택했습니다 와이드팬츠 소화하기 어렵네여
COMMENT	당근죠스	2021.10.02. 20:15	■디매인 라이프■	None	아무리 봐도 전 이쁘던데,, 아쉽네여
COMMENT	백구윈터	2021.10.02. 20:40	■디매인 라이프■	None	호불호가 심하면 그냥 보내주는 게 나은거 같습니다...
POSTING	백구윈터	2021.10.02. 19:26	■디매인 라이프■	문제의 청바지 입고 다시 코디해봤습니다	이렇게 코디해도 별론가요?  그럼 고민 없이 반품하겠습니다 ㅜㅜ
COMMENT	로건 리	2021.10.02. 19:27	■디매인 라이프■	None	바지 보내주세요 그냥
COMMENT	백구윈터	2021.10.02. 19:27	■디매인 라이프■	None	허헉
COMMENT	걷기대장	2021.10.02. 19:28	■디매인 라이프■	None	젊은 느낌으로 와이드 사신것 같은데 ㅠ 그냥 보내주세요 ㅠ
COMMENT	백구윈터	2021.10.02. 19:28	■디매인 라이프■	None	반품 신청 들어갑니다
COMMENT	187kg	2021.10.02. 19:29	■디매인 라이프■	None	신발 컨버스처럼 낮은거 신고 덮으면 이쁠거같은대요?? 전글보고왔는데 안이상한데..
COMMENT	백구윈터	2021.10.02. 19:30	■디매인 라이프■	None	허어 고민되네요 ㅠㅠ
COMMENT	백구윈터	2021.10.02. 19:30	■디매인 라이프■	None	컨버스 신고 다시 찍어볼게요 잠시만여
COMMENT	백구윈터	2021.10.02. 19:32	■디매인 라이프■	None
COMMENT	187kg	2021.10.02. 19:33	■디매인 라이프■	None	아 근데 길이도 좀 긴거같긴하네요 ㅠㅠ 밑단 넓이도 그렇게 안넓어서 덮히지도 않는거같구 ㅠㅠ
COMMENT	187kg	2021.10.02. 19:34	■디매인 라이프■